In [1]:
from kafka import KafkaAdminClient, KafkaProducer, KafkaConsumer

# admin

In [2]:
broker = "localhost:9092"
admin = KafkaAdminClient(bootstrap_servers=[broker])

In [3]:
admin.list_topics()

[]

In [4]:
from kafka.admin import NewTopic

In [5]:
# admin.delete_topics(["even_nums"])

In [6]:
from kafka.errors import TopicAlreadyExistsError

In [7]:
try:
    admin.create_topics([NewTopic(name="even_nums", num_partitions=1, replication_factor=1)])
except TopicAlreadyExistsError:
    print("already exists")

In [8]:
try:
    admin.create_topics([NewTopic(name="odd_nums", num_partitions=2, replication_factor=1)])
except TopicAlreadyExistsError:
    print("already exists")

In [9]:
admin.list_topics()

['even_nums', 'odd_nums']

# producer

In [10]:
producer = KafkaProducer(bootstrap_servers=[broker])

In [11]:
result = producer.send("even_nums", bytes(str(0), "utf-8"))

In [12]:
result

In [13]:
result.get()

RecordMetadata(topic='even_nums', partition=0, topic_partition=TopicPartition(topic='even_nums', partition=0), offset=0, timestamp=1700248443201, log_start_offset=0, checksum=None, serialized_key_size=-1, serialized_value_size=1, serialized_header_size=-1)

In [14]:
from threading import Thread, Lock

lock = Lock()
def Print(*args):
    with lock:
        print(*args)

Print("hi")

hi


In [15]:
import time, threading

def num_producer(topic, start, step):
    producer = KafkaProducer(bootstrap_servers=[broker])
    num = start
    while True:
        if num < 10:
            Print("send", num, "to", topic)
        producer.send(topic, bytes(str(num), "utf-8"))
        num += step
        time.sleep(1)

threading.Thread(target=num_producer, args=("even_nums", 0, 2)).start()
threading.Thread(target=num_producer, args=("odd_nums", 1, 2)).start()

send 0 to even_nums
send 1 to odd_nums
send 2 to even_nums
send 3 to odd_nums
send 4 to even_nums
send 5 to odd_nums
send 7 to odd_nums
send 6 to even_nums
send 9 to odd_nums
send 8 to even_nums


# consumer

In [16]:
consumer = KafkaConsumer(bootstrap_servers=[broker])

In [17]:
batch = consumer.poll(1000)
batch

{}

In [18]:
consumer.assignment()

set()

## Manual Partition Assignment

In [19]:
from kafka import TopicPartition

In [20]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
consumer.assign([TopicPartition("even_nums", 0)])
consumer.seek_to_beginning()
consumer.assignment()

{TopicPartition(topic='even_nums', partition=0)}

In [23]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

126
128
130
132
134


## Automatic Partition Assignment

In [24]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
consumer.subscribe(["even_nums"])
print(consumer.assignment())

set()


In [25]:
_ = consumer.poll(1000)
print(consumer.assignment())
consumer.seek_to_beginning()

{TopicPartition(topic='even_nums', partition=0)}


In [27]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

236
238
240


# Multiple Assignment

In [31]:
consumer = KafkaConsumer(bootstrap_servers=[broker])
consumer.subscribe(["even_nums", "odd_nums"])
print(consumer.assignment())

_ = consumer.poll(1000)
print(consumer.assignment())
consumer.seek_to_beginning()

set()
{TopicPartition(topic='odd_nums', partition=0), TopicPartition(topic='odd_nums', partition=1), TopicPartition(topic='even_nums', partition=0)}


In [34]:
batch = consumer.poll(1000)
for topic_partition, messages in batch.items():
    print(topic_partition)
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

TopicPartition(topic='even_nums', partition=0)
330
332
334
336
338
340
342
344
346
TopicPartition(topic='odd_nums', partition=0)
331
333
339
347
TopicPartition(topic='odd_nums', partition=1)
335
337
341
343
345


In [39]:
positions = {}
for tp in consumer.assignment():
    pos = consumer.position(tp)
    positions[tp] = pos
positions

{TopicPartition(topic='odd_nums', partition=0): 95,
 TopicPartition(topic='odd_nums', partition=1): 79,
 TopicPartition(topic='even_nums', partition=0): 175}

In [41]:
consumer2 = KafkaConsumer(bootstrap_servers=[broker])

In [44]:
consumer2.assign(positions.keys())

In [46]:
for tp, offset in positions.items():
    consumer2.seek(tp, offset)

# Take over for another Consumer

In [47]:
batch = consumer2.poll(1000)
for topic_partition, messages in batch.items():
    print(topic_partition)
    for msg in messages:
        print(int(str(msg.value, "utf-8")))

TopicPartition(topic='odd_nums', partition=0)
349
353
355
357
359
363
365
371
381
385
387
389
391
399
403
405
407
409
415
417
427
435
439
443
445
447
449
451
453
455
463
465
467
475
477
479
489
495
497
501
505
511
517
519
521
523
531
533
535
541
543
547
553
555
557
561
571
577
579
587
597
599
601
609
613
621
629
631
639
641
645
651
653
657
659
669
673
677
679
681
687
695
705
709
711
723
725
727
729
733
735
743
747
759
761
763
767
771
775
777
779
781
785
787
789
791
793
797
799
801
803
807
809
811
813
TopicPartition(topic='odd_nums', partition=1)
351
361
367
369
373
375
377
379
383
393
395
397
401
411
413
419
421
423
425
429
431
433
437
441
457
459
461
469
471
473
481
483
485
487
491
493
499
503
507
509
513
515
525
527
529
537
539
545
549
551
559
563
565
567
569
573
575
581
583
585
589
591
593
595
603
605
607
611
615
617
619
623
625
627
633
635
637
643
647
649
655
661
663
665
667
671
675
683
685
689
691
693
697
699
701
703
707
713
715
717
719
721
731
737
739
741
745
749
751
753
755
757


# Consumer Groups

In [54]:
def consume_odds(group, thread):
    consumer = KafkaConsumer(bootstrap_servers=[broker], group_id=group)
    consumer.subscribe("odd_nums")
    for i in range(10):   # TODO: forever
        batch = consumer.poll(1000)
        for tp, messages in batch.items():
            for msg in messages:
                Print(group, thread, msg.value)

threading.Thread(target=consume_odds, args=("g1", "t1")).start()
threading.Thread(target=consume_odds, args=("g2", "t2")).start()
threading.Thread(target=consume_odds, args=("g2", "t3")).start()

g1 t1 b'1629'
g1 t1 b'1631'
g1 t1 b'1635'
g1 t1 b'1637'
g1 t1 b'1639'
g1 t1 b'1641'
g1 t1 b'1647'
g1 t1 b'1649'
g1 t1 b'1633'
g1 t1 b'1643'
g1 t1 b'1645'
g2 t3 b'1629'
g2 t3 b'1631'
g2 t3 b'1635'
g2 t3 b'1637'
g2 t3 b'1639'
g2 t3 b'1641'
g2 t3 b'1647'
g2 t3 b'1649'
g1 t1 b'1651'
g2 t2 b'1633'
g2 t2 b'1643'
g2 t2 b'1645'
g2 t2 b'1651'
g2 t2 b'1653'
g1 t1 b'1653'
g2 t2 b'1655'
g1 t1 b'1655'
g2 t2 b'1657'
g1 t1 b'1657'
g1 t1 b'1659'
g2 t3 b'1659'
g1 t1 b'1661'
g2 t3 b'1661'
g2 t2 b'1663'
g1 t1 b'1663'
g2 t3 b'1665'
g1 t1 b'1665'
g2 t3 b'1667'
g1 t1 b'1667'
